# Leave-One-In Protocol

In [1]:
# Import necessary libraries
import copy
import csv
import json
import os
import sys
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Install the edist library using pip
# !pip install edist

# Import the modules from edist
import edist.ted as ted
import edist.sed as sed
import edist.aed as aed
import edist.dtw as dtw
import edist.seted as seted

# Other imports
from multiprocessing import Pool

In [2]:
insertion_cost = 1. # Cost of insertion
deletion_cost = 1. # Cost of deletion
leave_change = 1. # Cost of substitution
default_cost = 100 # Default cost 

start_edit = 1  # Define the starting edit value
end_edit = 21   # Define the ending edit value

num_cases = 300  # Define the number of cases to slice from the beginning of the list
casebase = "casebase300.json" # Store the casebase filename
directory = '' # Add the directory path

structural_similarity = ["Levenshtein Distance", "Affine Edit Distance", "Dynamic Time Warping", "Set Edit Distance", "Tree Edit Distance"]
explainerNames=["/Images/Anchors","/Images/Counterfactuals","/Images/GradCamTorch","/Images/IG", "/Images/LIME", "/Tabular/ALE", "/Tabular/Anchors","/Tabular/DeepSHAPGlobal", "/Tabular/DeepSHAPLocal", "/Tabular/DicePrivate","/Tabular/DicePublic","/Tabular/DisCERN","/Text/NLPClassifier","/Timeseries/CBRFox","/Tabular/IREX", "/Tabular/Importance", "/Text/LIME", "/Tabular/LIME", "/Tabular/NICE", "/Tabular/TreeSHAPGlobal", "/Tabular/TreeSHAPLocal", "/Tabular/KernelSHAPGlobal", "/Tabular/KernelSHAPLocal"]
control_nodes = np.array([["s", "p"]]) # Assume control_nodes is a 2-dimensional array

used_nodes = set()  # used nodes
unused_nodes = set() # unused nodes

start_time = time.time()

In [3]:
# Create a folder to store the random cases
folder_name = "BT_Random" 
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

random_bt_output = os.path.join(folder_name, "RandomBT.csv")

In [4]:
# delta: custom node distance function
def semantic_delta(x, y):
    if(x==y):
         ret = 0.
    elif(x!=None and y==None): #insertion
         ret = insertion_cost
    elif(x==None and y!=None): #deletion 
        ret = deletion_cost
    elif(x=='r'or y=='r'):  #we assign an infinite cost when comparing a root node
        ret = np.inf
    elif(x in ['s','p'] and y in['s','p']): #if both nodes are either sequence or priority, assign null cost
        ret = 0.
    elif(x in ['s','p'] or y in ['s','p']): #if one of the nodes is a sequence or priority, the other won't because of the previous rule
        ret = np.inf
    elif(x[0] == '/' and y[0]=='/'):
        ret =  leave_change  # substitution of explainer
    else:
        ret = leave_change  # substitution of explainer
    return ret

In [5]:
# Returns a list of explainers in the same order specified by the tree
def explainer_sequence(bt,node,adj,seq):
    seq.append(node)
    if adj: 
        for child in adj:
            explainer_sequence(bt, bt["nodes"][child],bt["adj"][child],seq)

In [6]:
# TED computation
def ted_similarity(q,c,delta):
    s1=[]
    explainer_sequence(q,q["nodes"][0],q["adj"][0],s1)
    s2=[]
    explainer_sequence(c,c["nodes"][0],c["adj"][0],s2)

    dist = ted.ted(q["nodes"], q["adj"], c["nodes"], c["adj"],delta)
    
    return  dist

In [7]:
# sequence edit distance of Levenshtein (1965)
def levenshtein_similarity(q,c, delta=None):
    s1=[]
    explainer_sequence(q,q["nodes"][0],q["adj"][0],s1)
    s2=[]
    explainer_sequence(c,c["nodes"][0],c["adj"][0],s2)

    dist = sed.sed(s1,s2, delta)
    
    return  dist


In [8]:
# sequence edit distance with affine gap costs using algebraic dynamic programming (ADP; Giegerich, Meyer, and Steffen, 2004),
# as applied by Paaßen, Mokbel, and Hammer (2016)
def aed_similarity(q,c, delta=None):
    s1=[]
    explainer_sequence(q,q["nodes"][0],q["adj"][0],s1)
    s2=[]
    explainer_sequence(c,c["nodes"][0],c["adj"][0],s2)
    dist = aed.aed(s1,s2,delta)
    
    return  dist

In [9]:
def default_dtw_distance(x,y):
    if x==y:
        return 0
    else: 
        return insertion_cost

In [10]:
# dynamic time warping distance of Vintsyuk (1968)    
def dtw_similarity(q,c, delta=None):
    s1=[]
    explainer_sequence(q,q["nodes"][0],q["adj"][0],s1)
    s2=[]
    explainer_sequence(c,c["nodes"][0],c["adj"][0],s2)
    if delta==None:
        delta = default_dtw_distance
    dist = dtw.dtw(s1,s2,delta)
    
    return  dist

In [11]:
# Hungarian algorithm of Kuhn, 1955 
def set_similarity(q,c, delta=None):
    s1=[]
    explainer_sequence(q,q["nodes"][0],q["adj"][0],s1)
    s2=[]
    explainer_sequence(c,c["nodes"][0],c["adj"][0],s2)
    dist = seted.seted(s1,s2,delta)
    
    return  dist

In [12]:
# Check if the node is a control_node
def is_control_node(node, node_index):
    # print('node:',node,'edge:',edge,'\n')
    if len(node) < 2 and node_index != 0:
        return True

    return False

In [13]:
# Get a node for substitution or replacement
def get_replacement_node(nodes, control_nodes):
    for i, control_set in enumerate(control_nodes):
        if control_set[0] in nodes:
            if len(control_set) == 1:
                continue
            if len(control_set) > 1 and len(control_set) >= 2 and control_set[1] not in nodes:
                return control_set[1]
        elif control_set[1] in nodes:
            if control_set[0] not in nodes:
                return control_set[0]
        else:
            return control_set[0]
    return control_nodes[-1][1]

In [14]:
# Get new node for edit operation
def get_new_node(nodes, used_nodes, operation):
    discard_nodes = ['r', 'f', 't']
    unused_nodes = [node for node in (set(nodes) - set(used_nodes)) if node not in discard_nodes]
    exp_nodes = list(set(explainerNames)) # in case of repeated explainers
    # exp_nodes = list(set(explainerNames) - set(nodes)) # in case of nonrepeated explainers
    # cont_nodes = list(set(control_nodes.flatten()) - set(nodes))

    if not unused_nodes:
        operation = 'insertion'
        new_node = random.choice(exp_nodes)
        node_index = None
        node = None  
        return operation, node_index, node, new_node

    if operation == 'replacement':
        unusednodes = [node for node in unused_nodes if node not in control_nodes]
        if not unusednodes:
            operation = 'insertion'
            new_node = random.choice(exp_nodes)
            node_index = None
            node = None
            return operation, node_index, node, new_node
        else:
            node = np.random.choice(unusednodes)
            node_index = nodes.index(node)
            new_node = ''

            if node[0] == '/':
                # node is an explainer
                if len(exp_nodes) > 0:
                    new_node = np.random.choice(exp_nodes)
                else:
                    new_node = None
            # else:
            #     # node is a control node
            #     unused_control_nodes = set(control_nodes.flatten()) - used_nodes
            #     print('\nunused_control_nodes:', unused_control_nodes)
            #     if len(unused_control_nodes) > 0:
            #         new_node = np.random.choice(control_nodes.flatten())
            #         print("c - new_node",new_node)
            #     else:
            #         print("New node")

    elif operation == 'deletion':
        discard_nodes = ['r', 'f', 't']
        unused_control_nodes = [node for node in nodes if not node.startswith('/') and node not in used_nodes and node not in discard_nodes]
        combined_nodes = unused_nodes + unused_control_nodes
        node = np.random.choice(combined_nodes)            
        node_index = nodes.index(node)
        new_node = ''

        if not unused_nodes:
            operation = 'insertion'
            operation, node_index, node, new_node = get_new_node(nodes, used_nodes, operation) 
        elif not unused_control_nodes:
            operation = np.random.choice(['replacement','insertion'])
            operation, node_index, node, new_node = get_new_node(nodes, used_nodes, operation)
        elif len(unused_control_nodes) == 1:
            operation = 'insertion'
            operation, node_index, node, new_node = get_new_node(nodes, used_nodes, operation)   
        else: 
            operation = 'deletion'  

    elif operation == 'insertion':
        new_node = random.choice(exp_nodes)
        node_index = None
        node = None  

    return operation, node_index, node, new_node

In [15]:
# Convert edge list to adjacency list
def ELtoAL(edges,nodes):       #converting edge list to adjacency list
    node_index,adj_dict = {},{}
    adj_list=[]
    for index, value in enumerate(nodes):
        node_index[value]=index
    for edge in edges:  
        u,v = edge
        u=node_index[edge[0]]
        v=node_index[edge[1]]
        if u not in adj_dict:
            adj_dict[u] = []
        if v not in adj_dict:
            adj_dict[v] = []
        adj_dict[u].append(v)

    for adj in list(adj_dict):
        adj_list.append(adj_dict[adj])

    return adj_list


In [16]:
# Convert adjacency list to edge list
def ALtoEL(nodes,adj): #converting adjacency list to edge list
    edgelist =[]
    node_ind,adj_index={},{}
    for index, value in enumerate(nodes):      
        for ind, val in enumerate(adj):   
            node_ind[index]=value
            adj_index[ind]=val

    for i in adj_index:
        for ad in adj_index[i]:
            if ad is not None: 
                u=node_ind[i]
                v=node_ind[ad]
                edge=(u,v)
                edgelist.append(edge)
            else:
                continue
    return edgelist

In [17]:
# Choose the random edit operation for random bt
def choose_random_operation(random_bt_prime, edits):
    nodes = random_bt_prime['nodes']
    adj = random_bt_prime['adj']
    edge = ALtoEL(nodes,adj)
    num_nodes = len(nodes)
    global num_operations
    num_operations = 0

    # initialize the children list for each node
    node_list = [{'id': node, 'children': []} for node in nodes]
    operation = ''
    
    if num_nodes >= 2:
        if num_nodes <= 3:
            operation = 'insertion'
        else:
            operation = np.random.choice(['replacement', 'insertion', 'deletion'])
        operation, node_index, node, new_node = get_new_node(nodes, used_nodes, operation)
        
        # if new_node not in nodes and new_node not in used_nodes and node not in used_nodes:
        if operation == 'replacement':
            # Perform the replacement
            if node[0] == '/':
                nodes[node_index] = new_node
            else:
                if node in control_nodes:
                    replacement_node = get_replacement_node(nodes, control_nodes)
                    nodes[nodes.index(node)] = replacement_node
                    edge = [(e[0] if e[0] != node else replacement_node, e[1] if e[1] != node else replacement_node) for e in edge]
                    random_bt_prime['edge'] = edge
        
        elif operation == 'deletion':  # Perform the deletion
            node = nodes[node_index]
            # node is not the root
            if node_index != 1:
                # Check whether it is a control node or explainer
                if is_control_node(node, edge):
                    print('node', node, adj)
                    parent_index = None
                    for i, sublist in enumerate(adj):
                        if node_index in sublist:
                            parent_node = nodes[i]
                            parent_index = i
                    # To delete a control node, move its children to the parent node of that control node 
                    if parent_index is None:
                        choose_random_operation(random_bt_prime, edits)
                    else:
                        # Append the children of the node to the parent node's adjacency list
                        adj[parent_index].extend(adj[node_index])
                        # Remove the adjacency list at node_index
                        del adj[node_index]
                        # Update the adjacency list to remove references to the deleted node and adjust indices
                        adj = [[idx-1 if idx > node_index else idx for idx in sublist if idx != node_index] for sublist in adj]
                        # Remove the node from the lists 
                        nodes.pop(node_index)
                        random_bt_prime['adj'] = adj
                    
                elif node[0] == '/':
                    # delete the node from the nodes list
                    del nodes[node_index]
                    # Remove the adjacency list at node_index
                    del adj[node_index]
                    adj = [[idx for idx in sublist if idx != node_index] for sublist in adj]
                    for i, neighbors in enumerate(adj):
                        updated_neighbors = [n - 1 if n > node_index else n for n in neighbors]
                        adj[i] = updated_neighbors
                    random_bt_prime['adj'] = adj
                else:
                    choose_random_operation(random_bt_prime, edits)
            else:
                print("Can't delete root node. Choose another node.")
                choose_random_operation(random_bt_prime, edits)

        elif operation == 'insertion':
            # new node is an explainer, insert as leaf node
            if new_node[0] == '/':
                discard_nodes = ['r', 'f', 't']
                c_nodes = list(filter(lambda node: not node.startswith('/') and node not in discard_nodes, nodes))
                parent_node = np.random.choice(list(set(c_nodes)))
                if parent_node in nodes:
                    parent_index = nodes.index(parent_node)
                    parent_adjacency_list = adj[parent_index]
                    if len(parent_adjacency_list) > 0:
                        first_node_index = parent_adjacency_list[0]
                        node_index_new = first_node_index
                        nodes.insert(node_index_new, new_node)
                        for i, node_adj in enumerate(adj):
                            adj[i] = [idx + 1 if idx > node_index_new - 1 else idx for idx in node_adj]
                        # Add the new_node index in front of the first node in adj[parent_index]
                        adj[parent_index].insert(0, node_index_new)
                        # Insert an empty list at the node_index_new position in the adjacency list
                        adj.insert(node_index_new, [])
                        # break
                    else:
                        print("The parent_adjacency_list is empty.")
                        choose_random_operation(random_bt_prime, edits)
                else:
                    node_index_new = len(nodes)
                    nodes.append(new_node)
                    # append new empty adjacency list to adj
                    adj.append([])
                    # update parent node's adjacency list
                    adj[parent_index].append(node_index_new)
                    adj[node_index_new].append(parent_index)
                    edge.append((parent_node, new_node))
                    adj = ELtoAL(edge,nodes)
                    random_bt_prime['adj'] = adj
            else:
                # new node is a control node, insert as root or parent node
                nodes.insert(0, new_node)
                adj.insert(0, [])
                for i in range(num_nodes-1):
                    for j, val in enumerate(adj[i]):
                        if val >= node_index:
                            adj[i][j] += 1
                edge.append((node, new_node))
    else:
        return random_bt_prime, edits, operation      
        
    print('Final - edits:',edits, 'random_bt_prime:', random_bt_prime)   
    print('*********************************************************\n')   

    end_time = time.time()
    computation_time = end_time - start_time
    print("\nComputation time:", computation_time, "seconds", ) 

    return random_bt_prime, edits, operation

In [18]:
# Compute the edit distance for the five structural metrics
def compute_edit_distance(random_index, random_bt, random_bt_dict, case_base, case_base_dict, structural_similarity):    
    # global explainer_dataframe
    random_bt = case_base[random_index]
    random_bt_dict[random_index]=random_bt
    random_bt_p = copy.deepcopy(random_bt)
    results = pd.DataFrame(columns=['Random BT', 'Edits', 'Operation', 'Modified BT', 'Case', 'Structural Similarity', 'Edit Distance'])
    
    # Loop to perform operations on the BT - maximum operation is the length of BT
    for edits in range(start_edit, end_edit):
        # Choose a random edit operation and get the new BT
        random_bt_prime, edits, operation = choose_random_operation(random_bt_p, edits)
        # For each structural_similarity, compute the edit distance using each similarity_metrics
        for algorithm in structural_similarity:
                # Compute the edit distance between the modified BT and all the BTs in the case base
                for bt_name, bt in case_base_dict.items():
                    # explainer_dataframe = pd.read_csv(metric,index_col=0)
                    if algorithm == "Tree Edit Distance":
                        score = ted_similarity(bt,random_bt_prime, delta=semantic_delta)
                    elif algorithm == "Levenshtein Distance":
                        score = levenshtein_similarity(bt,random_bt_prime, delta=semantic_delta)
                    elif algorithm == "Affine Edit Distance":
                        score = aed_similarity(bt,random_bt_prime, delta=semantic_delta)
                    elif algorithm == "Dynamic Time Warping":
                        score = dtw_similarity(bt,random_bt_prime, delta=semantic_delta)
                    elif algorithm == "Set Edit Distance":
                        score = set_similarity(bt,random_bt_prime, delta=semantic_delta)
                    
                    results.loc[len(results.index)] = [random_bt, edits, operation, random_bt_prime, bt, algorithm, score]
                    
        # Assign random_bt_prime to random_bt_p
        random_bt_p = copy.deepcopy(random_bt_prime)

    return results, random_bt_dict

In [19]:
# Load case base from json file
with open(casebase, "r") as f:
    case_base = json.load(f)

case_base_dict, random_bt_dict = {},{}
for i, bt in enumerate(case_base):
    case_base_dict[f'bt_{i}'] = bt

In [ ]:
# open the CSV file in append mode
with open(random_bt_output, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Random BT', 'Modified BT', 'Edits'])
    results = pd.DataFrame(columns=['Random BT', 'Edits', 'Operation', 'Modified BT', 'Case', 'Structural Similarity', 'Edit Distance'])
    ranking_results = pd.DataFrame(columns=['Random BT', 'Edits', 'Operation', 'Modified BT', 'Case', 'Structural Similarity', 'Edit Distance', 'Rank'])

    # Loop over the random BTs
    random_indices = []

    # create a sublist of the random BTs
    sublist = case_base[:num_cases]

    # create args_list with the random BTs in the sublist
    args_list = [(i, bt, random_bt_dict, case_base, case_base_dict, structural_similarity) for i, bt in enumerate(sublist)]
    
    # Create a pool of worker processes to run the function in parallel
    with Pool() as pool:
        print('Loading pool....')
    # Map the function over the list of arguments and get the results
        all_results = pool.starmap(compute_edit_distance, args_list)

    for i in range(len(all_results)):
        result = all_results[i][0]  # get the result at index i
        # convert the result list to a pandas dataframe and add it to the new_results dataframe
        random_bt_results = pd.DataFrame(result, columns=['Random BT', 'Edits', 'Operation', 'Modified BT', 'Case', 'Structural Similarity','Edit Distance'])
        results = pd.concat([results, random_bt_results])        
    file.close()

    # create a dictionary to store the data frames for each sheet
    sheet_dict = {}

    for i in range(len(all_results)):
        random_bt_dict = all_results[i][1]  # get the result_dict at index i

        for key, bt in random_bt_dict.items():
        # Rank the casebase based on edit distance for each metric
            # filter results by the current random_bt
            random_bt_results = results.loc[results['Random BT'] == random_bt_dict[key]]
            # # Group the results by edits and metric
            groups = random_bt_results.groupby(['Edits','Structural Similarity'])
            ranks = groups['Edit Distance'].rank(method='dense')
            # ranks = groups['Edit Distance'].rank(method='dense', ascending=False)
            # # Add the ranks to the results DataFrame
            random_bt_results['Rank'] = ranks
            # Add the filtered results to the sheet dictionary
            sheet_dict[key] = random_bt_results
            # save filtered results to a csv file
            random_bt_results.to_csv(directory + f'random_bt_{key}_results.csv', index=False)

In [ ]:
end_time = time.time()
computation_time = end_time - start_time
print("\nFinal Computation time:", computation_time, "seconds")